In [ ]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport

In [ ]:
cause_effect_table= pd.pivot_table(alert_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="is_global", margins=True, margins_name= 'Total')